<a href="https://colab.research.google.com/github/isabelacfs/home/blob/master/clickStreamIBM_original_30_11_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




#Entendendo interesses dos clientes pela análise de cliques do usuário



Outras fontes

 Understand customer interests with clickstream analysis

Dados:
https://github.com/IBM/db2-event-store-clickstream/blob/master/data/clickstream_data.csv


https://github.com/IBM/db2-event-store-clickstream/blob/master/README.md




Notebooks

https://nbviewer.org/github/IBM/db2-event-store-clickstream/tree/master/notebooks/

Outros notebooks de referência

https://www.kaggle.com/code/elielcm/quickstart

https://www.kaggle.com/code/kerneler/starter-netflix-audience-behaviour-ae7bda03-4


https://www.kaggle.com/datasets/fgarciablanco/data-clickstream-banco-galicia-2019

https://www.kaggle.com/datasets/adityawisnugrahas/eshop-clothing-dataset

https://www.kaggle.com/datasets/aeryss/clickstream-data-for-online-shopping

### Carregando e importando dados

In [53]:
from numpy import int64
import pandas as pd
from datetime import datetime
PATH = 'https://raw.githubusercontent.com/IBM/db2-event-store-clickstream/master/data/clickstream_data.csv'
df = pd.read_csv(PATH, dtype={'timestamp': int})

## Preparando os dados

#### Renomeando coluna

In [54]:

df.rename(columns={'Unnamed: 8': 'nomeDesconhecido'}, inplace=True)

#### Mostrando as primeiras linhas

In [55]:
df.head(10)

,eventId,eventType,timestamp,ipaddress,sessionId,userId,pageUrl,browser,nomeDesconhecido
0,20170522901,pageView,1496311260,169.34.56.78,y20170522a4499u21,ceaton,/www.cybershop.com,Chrome,1
1,20170522902,pageView,1496311320,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&action=catalog,Chrome,2
2,20170522903,pageView,1496311440,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&action=catalog,Chrome,1
3,20170522904,pageView,1496311500,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=A-pho...,Chrome,1
4,20170522905,pageView,1496311560,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=A-pho...,Chrome,1
5,20170522906,addToCart,1496311620,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=A-pho...,Chrome,1
6,20170522907,pageView,1496311680,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=S-pho...,Chrome,2
7,20170522908,pageView,1496311800,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=S-pho...,Chrome,3
8,20170522909,addToCart,1496311980,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&product=S-pho...,Chrome,1
9,20170522910,pageView,1496312040,169.34.56.78,y20170522a4499u21,ceaton,/estore?product_line=smartphones&action=catalog,Chrome,2


In [56]:
df.shape

(123, 9)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   eventId           123 non-null    int64 
 1   eventType         123 non-null    object
 2   timestamp         123 non-null    int64 
 3   ipaddress         123 non-null    object
 4   sessionId         123 non-null    object
 5   userId            123 non-null    object
 6   pageUrl           123 non-null    object
 7   browser           123 non-null    object
 8   nomeDesconhecido  123 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 8.8+ KB


### Preparando page URL

Encontramos uma / no link /www.cybershop.com e vamos retirar para que não fique diferente da outra. 

In [58]:
# encontrando todas

df.loc[df['pageUrl'].str.contains(r'/www.cybershop.com')].shape

(9, 9)

In [59]:
df['pageUrl'] = df['pageUrl'].str.strip('/')
df['pageUrl']

0                                      www.cybershop.com
1         estore?product_line=smartphones&action=catalog
2         estore?product_line=smartphones&action=catalog
3      estore?product_line=smartphones&product=A-phon...
4      estore?product_line=smartphones&product=A-phon...
                             ...                        
118    estore?product_line=smartphones&product=A-phon...
119    estore?product_line=smartphones&product=X-phon...
120    estore?product_line=smartphones&product=A-phon...
121    estore?product_line=smartphones&product=X-phon...
122                                    www.cybershop.com
Name: pageUrl, Length: 123, dtype: object

### Timestamp para data

</br>
Para esta conversão, primeiro, mudamos a coluna criamos uma nova coluna chamado timestamp to date e então convertemos a original para inteiro e, em seguida para string. Desta forma a conversão para datetime funcionou.

In [60]:
df['timestampToDate'] = df['timestamp'].astype(int).astype(str)
df.dtypes

eventId              int64
eventType           object
timestamp            int64
ipaddress           object
sessionId           object
userId              object
pageUrl             object
browser             object
nomeDesconhecido     int64
timestampToDate     object
dtype: object

Documentação do pandas sobre datetime
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html?highlight=to_datetime#pandas.to_datetime



</br>

A documentação nos diz que quando encontramos um time stamp com 10 dígitos, estamos diante de um formato que está na unidade em segundos. 


```
pd.to_datetime(1490195805, unit='s')

pd.to_datetime(1490195805433502912, unit='ns')
```




In [61]:
df['timestampToDate'] = pd.to_datetime(df['timestampToDate'], origin='unix', unit='s')
df['timestampToDate']

0     2017-06-01 10:01:00
1     2017-06-01 10:02:00
2     2017-06-01 10:04:00
3     2017-06-01 10:05:00
4     2017-06-01 10:06:00
              ...        
118   2017-06-22 14:09:00
119   2017-06-22 14:10:00
120   2017-06-22 14:22:00
121   2017-06-22 14:23:00
122   2017-06-22 14:27:00
Name: timestampToDate, Length: 123, dtype: datetime64[ns]

## Métricas



## Calculando o tempo gasto nas páginas

Calculamos a diferença do clique atual pelo anterior.

#### Duração na página



Time

In [62]:
delta = df['timestamp'] - df['timestamp'].shift()
df['time'] = delta

Preenchendo valores vazios com zero

In [63]:
# Preenche o valor NaN com zero
df['time'].fillna(0, inplace=True)

In [64]:
df['time']

0        0.0
1       60.0
2      120.0
3       60.0
4       60.0
       ...  
118     60.0
119     60.0
120    720.0
121     60.0
122    240.0
Name: time, Length: 123, dtype: float64

### Page hits

 Calcula page hits

Cria um dataframe separado com a quantidade de page hits por url visitada.

In [65]:
# cria um df com a quantidade de page hits por url visitada
dfCountHits = pd.DataFrame(df.loc[df.eventType == 'pageView', 'pageUrl'].value_counts())
dfCountHits.reset_index(inplace=True)
dfCountHits.rename(columns={'index': 'pageUrl', 'pageUrl': 'pageHits'}, inplace=True)

Criar df agrupado por url visitada juntamente com as demais métricas agregadas

In [66]:
# criar df agrupado por url visitada e as métricas agregadas
dfHits = pd.DataFrame(df.loc[df.eventType == 'pageView'].groupby('pageUrl').sum())
dfHits.reset_index(inplace=True)


### Mesclar (Inner ou merge)

Interseção entre o df com a métrica de hits por url e o df com as demais métricas agregadas

In [67]:
dfPageHits = pd.merge(dfHits, dfCountHits, on='pageUrl')


In [68]:
dfHits.shape
dfCountHits.shape

(46, 2)

In [69]:
dfPageHits.sort_values(by='pageHits', ascending=False)

,pageUrl,eventId,timestamp,nomeDesconhecido,time,pageHits
21,estore?product_line=smartphones&action=catalog,262216798076,19452102120,21,1260.0,13
45,www.cybershop.com,201705229574,14971281540,10,1810620.0,10
8,estore?product_line=computers&action=catalog,141193660847,10483320060,23,1320.0,7
22,estore?product_line=smartphones&product=A-phon...,141193660632,10477718400,10,480.0,7
43,estore?product_line=videogames&action=catalog,121023137894,8986992960,78,4320.0,6
36,estore?product_line=smartphones&product=S-phon...,100852614754,7486054740,11,660.0,5
25,estore?product_line=smartphones&product=A-phon...,100852614671,7481581380,11,420.0,5
38,estore?product_line=smartphones&product=S-phon...,80682091782,5988331980,9,480.0,4
34,estore?product_line=smartphones&product=M-phon...,80682091898,5990163000,7,480.0,4
24,estore?product_line=smartphones&product=A-phon...,60511569057,4494422280,3,840.0,3


Donwload de Page Hits

In [70]:
dfPageHits.to_excel('clickstreamPageHits.xlsx')

### PageView

Selecionando apenas aqueles eventos que são Page View 

Seleciona apenas os eventos de page view e soma o tempo gasto em cada página

In [71]:
# "select pageURL, count(*) as page_hits, sum(time) as total_time " +
#   "from ClickData where eventType='pageView' group by pageURL")
dfPageViewWithSeconds = df[df['eventType'] == 'pageView'].groupby(by=['pageUrl']).sum().sort_values(by='time', ascending=False)
dfPageViewWithSeconds


,eventId,timestamp,nomeDesconhecido,time
pageUrl,,,,
www.cybershop.com,201705229574,14971281540,10,1810620.0
estore?product_line=videogames&action=catalog,121023137894,8986992960,78,4320.0
estore?product_line=computers&action=catalog,141193660847,10483320060,23,1320.0
estore?product_line=smartphones&action=catalog,262216798076,19452102120,21,1260.0
estore?product_line=appliances&product=S-washer&action=details,20170522959,1496321220,10,900.0
estore?product_line=smartphones&product=A-phone&feature=camera&action=details,60511569057,4494422280,3,840.0
estore?product_line=smartphones&product=S-phone&action=details,100852614754,7486054740,11,660.0
estore?product_line=computers&product=L-top&action=details,60511568928,4492180980,6,540.0
estore?product_line=smartphones&product=S-phone&feature=camera&action=details,80682091782,5988331980,9,480.0


### AddToCart e Order

</br>

Vamos ver quanto tempo foi gasto nas páginas considerando os demais eventos que o usuário pode interagir nas páginas como adicionar ao carrinho e pedido.

In [72]:
# "select pageURL, count(*) as page_hits, sum(time) as total_time " +
#   "from ClickData where eventType='pageView' group by pageURL")
dfOtherEventsWithSeconds = df[df['eventType'] != 'pageView'].groupby(by=['eventType']).sum().sort_values(by='time', ascending=False)
dfOtherEventsWithSeconds



,eventId,timestamp,nomeDesconhecido,time
eventType,,,,
addToCart,221875752439,16465777680,22,2460.0
order,80682091767,5986586280,4,720.0


### Resumo do Tempo gasto  por página

In [73]:
df[['eventType', 'time']].groupby(by=['eventType']).sum().sort_values(by='time', ascending=False)


,time
eventType,
pageView,1827180.0
addToCart,2460.0
order,720.0


### Browser

Tempo gasto por navegador

In [74]:
df[['time', 'browser']].groupby(by=['browser']).sum().sort_values(by='time', ascending=False)


,time
browser,
Safari,1755420.0
Chrome,69600.0
Firefox,5340.0


 Eventos por navegador

In [75]:
df[['browser', 'eventType', 'time']].groupby(by=['browser', 'eventType']).sum().sort_values(by='time', ascending=False)


# df.groupby(by=['browser', 'eventType']).sum().sort_values(by='time_seconds', ascending=False)


time
browser eventType           
Safari  pageView   1754580.0
Chrome  pageView     68400.0
Firefox pageView      4200.0
        addToCart     1020.0
Chrome  addToCart      840.0
Safari  addToCart      600.0
Chrome  order          360.0
Safari  order          240.0
Firefox order          120.0

### Usuário

In [76]:
df[['userId', 'time']].groupby(by=['userId']).sum().sort_values(by='time', ascending=False)

,time
userId,
datkins,1747140.0
sanne,72180.0
cwang,4200.0
mtwain,3000.0
tchoo,1260.0
mking,1200.0
ceaton,1140.0
sraman,240.0


## Extraindo métricas por produto


In [77]:
df.head(5)

,eventId,eventType,timestamp,ipaddress,sessionId,userId,pageUrl,browser,nomeDesconhecido,timestampToDate,time
0,20170522901,pageView,1496311260,169.34.56.78,y20170522a4499u21,ceaton,www.cybershop.com,Chrome,1,2017-06-01 10:01:00,0.0
1,20170522902,pageView,1496311320,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,2,2017-06-01 10:02:00,60.0
2,20170522903,pageView,1496311440,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,1,2017-06-01 10:04:00,120.0
3,20170522904,pageView,1496311500,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:05:00,60.0
4,20170522905,pageView,1496311560,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:06:00,60.0


#### Extrair nome do produto e armazenar numa coluna separado

Fonte: https://www.w3schools.com/python/python_regex.asp

https://www.w3resource.com/pandas/series/series-str-findall.php

In [78]:
df['pageUrl'].value_counts()

estore?product_line=smartphones&action=catalog                                            13
www.cybershop.com                                                                         10
estore?product_line=smartphones&product=A-phone&action=details                             7
estore?product_line=computers&action=catalog                                               7
estore?product_line=videogames&action=catalog                                              6
estore?product_line=smartphones&product=A-phone&feature=color&action=details               5
estore?product_line=smartphones&product=S-phone&action=details                             5
estore?product_line=smartphones&product=A-phone&action=addToCart                           4
estore?product_line=smartphones&product=S-phone&feature=camera&action=details              4
estore?product_line=smartphones&product=M-phone&action=details                             4
estore?product_line=smartphones&product=A-phone&feature=camera&action=

## Cria a coluna de linha de produto

In [79]:
df['product_line'] = ''

### Adiciona os nomes das respectivas linhas de produtos

In [80]:
# df.loc[df['faturamento']=='15k', 'euro'] = df.loc[df['faturamento']=='15k', 'euro'].replace(0,1)
df.loc[df['pageUrl'].str.contains('smartphones'), 'product_line'] = 'smartphone'
df.loc[df['pageUrl'].str.contains('computers'), 'product_line'] = 'computers'
df.loc[df['pageUrl'].str.contains('videogames'), 'product_line'] = 'videogames'
df.loc[df['pageUrl'].str.contains('headphones'), 'product_line'] = 'headphones'
df.loc[df['pageUrl'].str.contains('appliances'), 'product_line'] = 'appliances'
df.loc[df['pageUrl'].str.contains('hometheater'), 'product_line'] = 'hometheater'
df.loc[df['pageUrl'].str.contains('cameras'), 'product_line'] = 'cameras'
df.loc[df['pageUrl'].str.contains('carelectronics'), 'product_line'] = 'carelectronics'

### Product coluna

In [81]:
df['product'] = ''
# phones
df.loc[df['pageUrl'].str.contains('A-phone'), 'product'] = 'A-phone'
df.loc[df['pageUrl'].str.contains('S-phone'), 'product'] = 'S-phone'
df.loc[df['pageUrl'].str.contains('L-phone'), 'product'] = 'L-phone'
df.loc[df['pageUrl'].str.contains('X-phone'), 'product'] = 'X-phone'
df.loc[df['pageUrl'].str.contains('H-phone'), 'product'] = 'H-phone'
df.loc[df['pageUrl'].str.contains('M-phone'), 'product'] = 'M-phone'

df.loc[df['pageUrl'].str.contains('M-top'), 'product'] = 'M-top'
df.loc[df['pageUrl'].str.contains('G-washer'), 'product'] = 'G-washer'
df.loc[df['pageUrl'].str.contains('T-lenses'), 'product'] = 'T-lenses'
df.loc[df['pageUrl'].str.contains('Y-receiver'), 'product'] = 'Y-receiver'
df.loc[df['pageUrl'].str.contains('W-game'), 'product'] = 'W-game'

df.loc[df['pageUrl'].str.contains('L-top'), 'product'] = 'L-top'
df.loc[df['pageUrl'].str.contains('S-dryer'), 'product'] = 'S-dryer'
df.loc[df['pageUrl'].str.contains('S-washer'), 'product'] = 'S-washer'
df.loc[df['pageUrl'].str.contains('A-head'), 'product'] = 'A-head'
df.loc[df['pageUrl'].str.contains('B-head'), 'product'] = 'B-head'
df.loc[df['pageUrl'].str.contains('H-top'), 'product'] = 'H-top'
df.loc[df['pageUrl'].str.contains('D-top'), 'product'] = 'D-top'
df.loc[df['pageUrl'].str.contains('D-head'), 'product'] = 'D-head'
df.loc[df['pageUrl'].str.contains('B-theater'), 'product'] = 'B-theater'
df.loc[df['pageUrl'].str.contains('ear-phone'), 'product'] = 'ear-phone'



### Action

In [82]:
df['action'] = ''

df.loc[df['pageUrl'].str.contains('catalog'), 'action'] = 'catalog'
df.loc[df['pageUrl'].str.contains('details'), 'action'] = 'details'
df.loc[df['pageUrl'].str.contains('addToCart'), 'action'] = 'addToCart'
df.loc[df['pageUrl'].str.contains('order'), 'action'] = 'order'

### Feature

In [83]:
df['feature'] = ''


df.loc[df['pageUrl'].str.contains('color'), 'feature'] = 'color'
df.loc[df['pageUrl'].str.contains('camera'), 'feature'] = 'camera'
df.loc[df['pageUrl'].str.contains('battery'), 'feature'] = 'battery'
df.loc[df['pageUrl'].str.contains('processor'), 'feature'] = 'processor'
df.loc[df['pageUrl'].str.contains('voice_assistant'), 'feature'] = 'voice_assistant'

### Shopping cart

In [84]:
df['shopping_cart'] = ''

df.loc[df['pageUrl'].str.contains('c20170622x111'), 'shopping_cart'] = 'c20170622x111'
df.loc[df['pageUrl'].str.contains('c20170622x122'), 'shopping_cart'] = 'c20170622x122'
df.loc[df['pageUrl'].str.contains('c20170622x422'), 'shopping_cart'] = 'c20170622x422'


In [85]:
df

,eventId,eventType,timestamp,ipaddress,sessionId,userId,pageUrl,browser,nomeDesconhecido,timestampToDate,time,product_line,product,action,feature,shopping_cart
0,20170522901,pageView,1496311260,169.34.56.78,y20170522a4499u21,ceaton,www.cybershop.com,Chrome,1,2017-06-01 10:01:00,0.0,,,,,
1,20170522902,pageView,1496311320,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,2,2017-06-01 10:02:00,60.0,smartphone,,catalog,,
2,20170522903,pageView,1496311440,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&action=catalog,Chrome,1,2017-06-01 10:04:00,120.0,smartphone,,catalog,,
3,20170522904,pageView,1496311500,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:05:00,60.0,smartphone,A-phone,details,,
4,20170522905,pageView,1496311560,169.34.56.78,y20170522a4499u21,ceaton,estore?product_line=smartphones&product=A-phon...,Chrome,1,2017-06-01 10:06:00,60.0,smartphone,A-phone,details,color,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,20170523019,pageView,1498140540,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=A-phon...,Safari,1,2017-06-22 14:09:00,60.0,smartphone,A-phone,details,camera,
119,20170523020,pageView,1498140600,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=X-phon...,Safari,12,2017-06-22 14:10:00,60.0,smartphone,X-phone,details,,
120,20170523021,pageView,1498141320,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=A-phon...,Safari,1,2017-06-22 14:22:00,720.0,smartphone,A-phone,details,camera,
121,20170523022,pageView,1498141380,119.14.137.99,y20170530a5463u14,sanne,estore?product_line=smartphones&product=X-phon...,Safari,4,2017-06-22 14:23:00,60.0,smartphone,X-phone,addToCart,,


# Extraindo os parametros da url page via regex

#### Calculando as métricas por cada tipo de linha de produto

In [86]:
# # filtrando por product_line

df.groupby(by='product_line').sum().sort_values(by='time', ascending=False)


,eventId,timestamp,nomeDesconhecido,time
product_line,,,,
,282387321341,20957867820,14,1811340.0
smartphone,1290913469370,95811510240,115,7140.0
videogames,161364183810,11981378040,82,4980.0
computers,322728367616,23961259920,47,2640.0
appliances,100852614725,7481590560,37,1560.0
headphones,161364183877,11983120680,28,1200.0
hometheater,80682091867,5988908760,14,840.0
carelectronics,40341045865,2992631580,7,360.0
cameras,40341045855,2992629720,6,300.0


## Criar um tabelão

Utilizar page hits + df com base na chave event id ou page url

In [87]:
cs = pd.merge(df, dfPageHits, on='pageUrl', how='outer')
cs.shape

(123, 21)

In [88]:
# cria nova tabela apenas com colunas úteis

cs.columns

Index(['eventId_x', 'eventType', 'timestamp_x', 'ipaddress', 'sessionId',
       'userId', 'pageUrl', 'browser', 'nomeDesconhecido_x', 'timestampToDate',
       'time_x', 'product_line', 'product', 'action', 'feature',
       'shopping_cart', 'eventId_y', 'timestamp_y', 'nomeDesconhecido_y',
       'time_y', 'pageHits'],
      dtype='object')

In [89]:
clickstream = pd.DataFrame(cs[['eventId_x', 'eventType', 'timestamp_x','timestampToDate', 'ipaddress', 'sessionId', 'userId', \
                               'pageUrl', 'browser', 'nomeDesconhecido_x', 'time_x', 'product_line', 'product', \
                               'action', 'feature', 'shopping_cart', 'pageHits']])

#### Download da tabela em excel

In [90]:
hoje = datetime.today()

In [91]:
clickstream.to_excel('clickstream-{0}.xlsx'.format(hoje))